In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
from bs4 import BeautifulSoup
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import chromedriver_binary

# 1.Data Collection

In [3]:
data = pd.read_csv('Raw_data/products_url_with_name_sephora.csv', index_col = 0)

In [4]:
data.head()

,category,CSS-class,url,classification,sku,p_id,product_name
0,moisturizing-cream-oils-mists,css-1o0t476,https://www.sephora.com/product/ultra-repair-c...,product,1217744,P248407,ultra-repair-cream-intense-hydration
1,moisturizing-cream-oils-mists,css-1o0t476,https://www.sephora.com/product/water-drench-h...,product,1898550,P415701,water-drench-hyaluronic-cloud-cream
2,moisturizing-cream-oils-mists,css-1o0t476,https://www.sephora.com/product/ceramidin-tm-c...,product,2077840,P434363,ceramidin-tm-cream
3,moisturizing-cream-oils-mists,css-1o0t476,https://www.sephora.com/product/ultra-facial-c...,product,2172526,P421996,ultra-facial-cream
4,moisturizing-cream-oils-mists,css-1o0t476,https://www.sephora.com/product/confidence-in-...,product,1868256,P411403,confidence-in-cream-transforming-moisturizing-...


In [5]:
# credit this website for this code: https://www.peterbe.com/plog/best-practice-with-retries-with-requests
# best practice to catch server errors and try

def requests_retry_session(
    # if the backoff_factor is 0.1, then sleep() will sleep for [0.05s, 0.1s, 0.2s, 0.4s, ...] between retries. 
    # It will also force a retry if the status code returned is 500, 502, 503 or 504.
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 503, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [6]:
#this code creates a function that gets the browser to scroll down
def scrollDown(driver, n_scroll):
    body = driver.find_element(By.TAG_NAME ,"body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to the bottom of the page

In [ ]:
test_url = data['url'][0]

try:
    # simply input the url takes a long time
    # it seems like the user only allows specific user-agent string, adding this headers helps
    response = requests_retry_session().get(
        test_url, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    )
except Exception as x:
    print('It failed :(', x.__class__.__name__)
else:
    print('It eventually worked', response.status_code)


In [ ]:
# simply input the url takes a long time
# it seems like the user only allows specific user-agent string, adding this headers helps
r = requests.get('https://www.sephora.com', headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})

In [ ]:
#it took a few days to scrape this since there were so many URLs to scrape and after a while it would throw a
#HTTPSConnectionPool(host='www.sephora.com', port=443): Max retries exceeded with url error


#for i in range(1310, 2768):
#for i in range(681, 2767):
for i in urls.index: #for every URL in my urls dataframe
    url = urls.URL[i]
    res = requests.get(url)
    time.sleep(10) #taking a break before moving on so that Sephora doesn't block
    #if request fails print the error
    if res.status_code != 200:
        print('error when requestion {}'.format(url))
        break
        
    soup = BeautifulSoup(res.content, 'html.parser') #getting the soup of the requested URL data
    
    path = "./data/soups/soup"+str(i)+".html"
    
    with open(path, 'w') as f: #saving soup as an .html file for later use
        f.write(str(soup))

In [ ]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
soup

In [ ]:
soup.find('title').text

In [ ]:
def initiate_ChromeDriver_and_BeautifulSoup(single_url):
    driver = webdriver.Chrome()
    driver.get(single_url) 
    
    time.sleep(10) # let the page load
        
    browser = scrollDown(driver, 20) #scroll down the page
    time.sleep(20) # let the whole page load
    
    res = requests.get(single_url, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
    soup = BeautifulSoup(response.text, 'html.parser')
    
    brand_name = driver.find_elements(By.CLASS_NAME, "css-1gyh3op eanm77i0").text
    product_name = driver.find_element(By.CLASS_NAME, "css-1pgnl76").text
    driver.close()
    
    return brand_name
    

In [10]:
test_url = 'https://www.sephora.com/product/farmacy-honey-halo-ultra-hydrating-ceramide-moisturizer-P463036?skuId=2527638'
driver = webdriver.Chrome()
driver.get(test_url) 
    
time.sleep(20) # let the page load
        
browser = scrollDown(driver, 20) #scroll down the page
time.sleep(40) # let the whole page load
    
res = requests.get(test_url, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
soup = BeautifulSoup(res.text, 'html.parser')
    
brand_name = driver.find_element(By.CLASS_NAME, "css-1gyh3op").text
product_name = driver.find_element(By.CLASS_NAME, "css-1pgnl76").text
size = driver.find_elements(By.CLASS_NAME,"css-8llsux")[0].text
price = driver.find_elements(By.CLASS_NAME,"css-0")[4].text
no_reviews = driver.find_element(By.CLASS_NAME,"css-1coslxg").text
no_loves = driver.find_element(By.CLASS_NAME,"css-jk94q9").text 
temp = driver.find_elements(By.CLASS_NAME,"css-1ac1x0l")
avg_ratings = temp[0].text 
perc_reccommended = temp[1].text



In [11]:
driver.find_element(By.CLASS_NAME, "css-1gyh3op").text

'Farmacy'

In [12]:
driver.find_element(By.CLASS_NAME, "css-1pgnl76").text

'Honey Halo Ultra-Hydrating Ceramide Moisturizer'

In [13]:
driver.find_elements(By.CLASS_NAME,"css-8llsux")[0].text

'3.4 oz/ 100 mL\nNEW'

In [14]:
driver.find_elements(By.CLASS_NAME,"css-0")[4].text # for the complexity of this work, we will only scrap price of 1 variant


'$68.00'

In [15]:
driver.find_element(By.CLASS_NAME,"css-1coslxg").text 

'683'

In [16]:
driver.find_element(By.CLASS_NAME,"css-jk94q9").text 


'58.9K'

In [17]:
driver.find_elements(By.CLASS_NAME,"css-1ac1x0l")[1].text

'86%'

In [70]:
# using this to look for ingredients doesnt work
print(driver.find_elements(By.CLASS_NAME,"css-o44is")[1])


IndexError: list index out of range

In [19]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="css-mao3d8" lang="en"><head data-comp="Head "><title>Honey Halo Ultra-Hydrating Ceramide Moisturizer - Farmacy | Sephora</title><script importance="low" src="https://www.sephora.com/resources/3203a2234e642a03f9e3b5f9aca755a11630011008b74" type="text/javascript"></script><script>if (typeof global === "undefined") window.global = window;global.Sephora = global.Sephora || {};Sephora.Util = {};Sephora.Util.Perf = { loadEvents: [] };Sephora.Util.Perf.isReportSupported = function () {    return window.performance && typeof window.performance.mark === "function";};if(Sephora.Util.Perf.isReportSupported()) window.performance.mark("HeadscriptRuntime Start");</script><meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/><meta content="Shop Farmacy‘s Honey Halo Ultra-Hydrating Ceramide Moisturizer at Sephora. This moisturizer replenishes dry skin and visibly plumps. " id="metaDescript

In [27]:
soup.title.string

'Honey Halo Ultra-Hydrating Ceramide Moisturizer - Farmacy | Sephora'

In [29]:
print(soup.body.contents)

[<script>if(Sephora.Util.Perf.isReportSupported()) {   window.performance.mark("HeadscriptRuntime End");   window.performance.measure("HeadscriptRuntime", "HeadscriptRuntime Start", "HeadscriptRuntime End");}</script>, <div class="css-o44is"><div rootid="Header" style="display:contents"><header style="display:contents"><div class="css-1j3fzx1" data-comp="TestTarget "><div class="css-t444sn"><div class="css-1q0bpb9"></div><div class="css-rzwxtl"><div class="css-1ehk86j"></div><div class="css-1wftfeq"></div></div></div></div><div class="css-56pq4n"><div class="css-wxytes e65zztl0" data-comp="StyledComponent BaseComponent "><div class="css-17rgt6"><a class="css-cdhtj" href="/"><img alt="Sephora" class="css-5kzluc e65zztl0" data-at="sephora_logo_ref" data-comp="Image StyledComponent BaseComponent " src="/img/ufe/logo.svg"/></a><div class="css-817iol"><div class="css-79elbk" data-comp="SiteSearch "><form class="css-7boy05" role="search"><label class="css-vkf4am" for="site_search_input" id="

In [82]:
print(soup.body.contents[8])# if we dont wrap it up in a list it would give an iterator object or a generator

<script data-comp="PageJSON " id="linkStore" type="text/json">{"page":{"product":{"breadcrumbsSeoJsonLd":"{\"itemListElement\":[{\"item\":{\"name\":\"Skincare\",\"@id\":\"https://www.sephora.com/shop/skincare\"},\"@type\":\"ListItem\",\"position\":1},{\"item\":{\"name\":\"Moisturizers\",\"@id\":\"https://www.sephora.com/shop/moisturizing-cream-oils-mists\"},\"@type\":\"ListItem\",\"position\":2},{\"item\":{\"name\":\"Moisturizers\",\"@id\":\"https://www.sephora.com/shop/moisturizer-skincare\"},\"@type\":\"ListItem\",\"position\":3}],\"@type\":\"BreadcrumbList\",\"@context\":\"http://schema.org\"}","combinedProductBanner":[{"componentName":"Sephora RWD Banner Component","componentType":93,"enableTesting":false,"imageAltText":"Clean + Planet Positive Beauty","imageLargeViewPortHeight":"5","imageLargeViewPortSource":"/contentimages/test/wendalynn/apppadding/640x5.jpg","imageLargeViewPortWidth":"640","imageSmallViewPortHeight":"103","imageSmallViewPortSource":"/contentimages/ppagebanners/0

In [164]:
print(list(soup.body.contents[8])[0])

{"page":{"product":{"breadcrumbsSeoJsonLd":"{\"itemListElement\":[{\"item\":{\"name\":\"Skincare\",\"@id\":\"https://www.sephora.com/shop/skincare\"},\"@type\":\"ListItem\",\"position\":1},{\"item\":{\"name\":\"Moisturizers\",\"@id\":\"https://www.sephora.com/shop/moisturizing-cream-oils-mists\"},\"@type\":\"ListItem\",\"position\":2},{\"item\":{\"name\":\"Moisturizers\",\"@id\":\"https://www.sephora.com/shop/moisturizer-skincare\"},\"@type\":\"ListItem\",\"position\":3}],\"@type\":\"BreadcrumbList\",\"@context\":\"http://schema.org\"}","combinedProductBanner":[{"componentName":"Sephora RWD Banner Component","componentType":93,"enableTesting":false,"imageAltText":"Clean + Planet Positive Beauty","imageLargeViewPortHeight":"5","imageLargeViewPortSource":"/contentimages/test/wendalynn/apppadding/640x5.jpg","imageLargeViewPortWidth":"640","imageSmallViewPortHeight":"103","imageSmallViewPortSource":"/contentimages/ppagebanners/072221/2020-07-22-july-clean+planetpositive-site-desktop-produc

In [147]:
print(list(soup.body.div)[1].div.main.contents[0].next_sibling) # inside main

<div class="css-1hd1tp0" data-comp="BackToTopButton " style="visibility:hidden;opacity:0;transform:translateY(-100%)"><button class="css-nzzcrh e65zztl0" data-at="back_to_top" data-comp="StyledComponent BaseComponent " type="button">Back to Top</button></div>


In [155]:
print(list(soup.body.div)[1].div.main.contents[0].contents[1]) #inside regular product but cant find our class

<div class="css-9q6f95 e65zztl0" data-comp="StyledComponent BaseComponent "><div><h1 class="css-11zrkxf e65zztl0" data-comp="DisplayName StyledComponent BaseComponent "><a class="css-1gyh3op e65zztl0" data-at="brand_name" data-comp="StyledComponent BaseComponent " href="/brand/farmacy?icid2=product_link_brand">Farmacy</a><br/><span class="css-1pgnl76 e65zztl0" data-at="product_name" data-comp="StyledComponent BaseComponent ">Honey Halo Ultra-Hydrating Ceramide Moisturizer</span></h1><div class="css-y55er9 e65zztl0" data-comp="Flex StyledComponent BaseComponent "><a class="css-whkkt7 e65zztl0" data-comp="ReviewsAnchor StyledComponent BaseComponent " href="#ratings-reviews-container"><div aria-label="4.5 stars" class="css-jp4jy6" data-comp="StarRating "><div><span class="css-92vnvr"></span><span class="css-m88no0"></span><span class="css-m88no0"></span><span class="css-m88no0"></span><span class="css-m88no0"></span></div><div class="css-j7llew" data-at="star_rating_style" style="width:88

In [ ]:
class Product_Info:
    def __init__(self, starting_value, product_list):
        self.starting_value=starting_value
        self.product_list=product_list
        
        
    def load_selenium_beautifulsoup(self, product_link):
        # load selenium and beautifulsoup
        driver.get(product_link)

        response = requests.get(product_link, )
        self.soup = BeautifulSoup(response.text, 'html.parser')

    def scrolling_to_product_type(self):
        while not driver.find_elements_by_class_name('css-dvzm2b')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-dvzm2b')[0])
            sleep(2)
        return
    
        
    def scrolling_to_id(self):
        if not driver.find_elements_by_class_name('css-v7k1z0')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-v7k1z0')[0])
            sleep(2)
        return

    def scrolling_to_price(self):
        if not driver.find_elements_by_class_name('css-slwsq8')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-slwsq8')[0])
            sleep(2)
        return
    
    def scrolling_to_reviews(self):
        while not driver.find_elements_by_class_name('css-fkgzyo') and not driver.find_elements_by_class_name('css-2ijuk8'):
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_element_by_xpath("//div[@id='ratings-reviews']"))
            sleep(3)
            print('signal')
        return
    
    def scrolling_to_skin_type(self):
        if not driver.find_elements_by_class_name('css-pz80c5')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-pz80c5')[0])
            sleep(2)
        return
    
    def substring_in_string(self, substring):  # update substring
        if substring.lower() in self.skin_description.lower():
            return 1
        else:
            return 0
        


    def get_info(self, i, product_link):
        self.load_selenium_beautifulsoup(product_link)
        if driver.current_url=='https://www.sephora.com/search?keyword=productnotcarried':
            return
        
        else:
            # product info
            self.scrolling_to_product_type()
            if len(driver.find_elements_by_class_name('css-dvzm2b'))>1:
                product_type_1[i]=driver.find_elements_by_class_name('css-dvzm2b')[1].text
            
            product_type_2[i]=driver.find_elements_by_class_name('css-lrl8sh')[0].text
            brand_name[i]=driver.find_elements_by_class_name('css-euydo4')[0].text
            product_name[i]=driver.find_elements_by_class_name('css-140z8k4')[0].find_elements(By.TAG_NAME, "span")[1].text

            self.scrolling_to_id()
            item_id[i]=int(re.findall("\d+", driver.find_elements_by_class_name('css-v7k1z0')[0].text)[0])

            self.scrolling_to_price()
            price[i]=float(re.findall(r"[-+]?[0-9]*\.?[0-9]+", driver.find_elements_by_class_name('css-slwsq8')[0].text)[0])
            number_of_loves[i]=int(self.soup.find('div', class_='css-jtf0an').find('span',class_='css-2rg6q7').span.text)


            # skin concerns/ types
            self.scrolling_to_skin_type()
            self.skin_description=driver.find_elements_by_class_name('css-pz80c5')[0].text

            skin_type_normal[i]=self.substring_in_string('Normal')
            skin_type_dry[i]=self.substring_in_string('Dry')
            skin_type_combination[i]=self.substring_in_string('Combination')
            skin_type_oily[i]=self.substring_in_string('Oily')
            skin_type_sensitive[i]=self.substring_in_string('Sensitive')
            skin_concerns_pores[i]=self.substring_in_string('Pores')
            skin_concerns_dark_spots[i]=self.substring_in_string('Dark Spots')
            skin_concerns_firmness_elasticity[i]=self.substring_in_string('Loss of firmness and elasticity')
            skin_concerns_dryness[i]=self.substring_in_string('Dryness')
            skin_concerns_oiliness[i]=self.substring_in_string('Oiliness')
            skin_concerns_puffiness[i]=self.substring_in_string('Puffiness')
            skin_concerns_redness_irritation[i]=self.substring_in_string('Normal')

            if 'dullness' in self.skin_description.lower() or 'uneven' in self.skin_description.lower():
                skin_concerns_dullness_uneven[i]=1
            else:
                skin_concerns_dullness_uneven[i]=0

            if 'fine lines' in self.skin_description.lower() or 'wrinkles' in self.skin_description.lower():
                skin_concerns_fine_lines_wrinkles[i]=1
            else:
                skin_concerns_fine_lines_wrinkles[i]=0

            # review info
            self.scrolling_to_reviews()
            print('signal2')


            if driver.find_elements_by_class_name('css-2ijuk8'):
                print('signal3')
                return
            else:
                number_of_reviews[i]=int(re.findall("\d+", driver.find_elements_by_class_name('css-tc6qfq')[0].text)[0])
                star_votes = driver.find_elements_by_class_name('css-960eb6')[0].find_elements(By.TAG_NAME, "tr") # get all of the rows in the table

                # Get votes for different stars    
                col = star_votes[0].find_elements(By.TAG_NAME, "td")[2].text 
                five_star[i]=int(col)

                col = star_votes[1].find_elements(By.TAG_NAME, "td")[2].text 
                four_star[i]=int(col)

                col = star_votes[2].find_elements(By.TAG_NAME, "td")[2].text 
                three_star[i]=int(col)

                col = star_votes[3].find_elements(By.TAG_NAME, "td")[2].text 
                two_star[i]=int(col)

                col = star_votes[4].find_elements(By.TAG_NAME, "td")[2].text 
                one_star[i]=int(col)

        
    def go_through_product_list(self):
        for i in range(len(self.product_list)):
            try:
                sleep(randint(2,30))
                print(i, self.product_list[i])
                self.get_info(self.starting_value+i, self.product_list[i]) 
                print(five_star[i])
        
            except:
                pass 
        return 

In [ ]:
brandname: 
<a data-at="brand_name" href="/brand/first-aid-beauty?icid2=product_link_brand" class="css-1gyh3op eanm77i0" data-comp="StyledComponent BaseComponent ">First Aid Beauty</a>
<a data-at="brand_name" href="/brand/skinfix?icid2=product_link_brand" class="css-1gyh3op eanm77i0" data-comp="StyledComponent BaseComponent ">Skinfix</a>
<a data-at="brand_name" href="/brand/fresh?icid2=product_link_brand" class="css-1gyh3op eanm77i0" data-comp="StyledComponent BaseComponent ">fresh</a>

In [9]:
soup = BeautifulSoup(res.text, 'html.parser')
    
brand_name = driver.find_element(By.CLASS_NAME, "css-1gyh3op").text
product_name = driver.find_element(By.CLASS_NAME, "css-1pgnl76").text
size = driver.find_elements(By.CLASS_NAME,"css-8llsux")[0].text
price = driver.find_elements(By.CLASS_NAME,"css-0")[4].text
no_reviews = driver.find_element(By.CLASS_NAME,"css-1coslxg").text
no_loves = driver.find_element(By.CLASS_NAME,"css-jk94q9").text 
temp = driver.find_elements(By.CLASS_NAME,"css-1ac1x0l")
avg_ratings = temp[0].text 
perc_reccommended = temp[1].text

In [ ]:
hightlights:

<h2 class="css-1lzcb6 eanm77i0" data-comp="StyledComponent BaseComponent ">Highlights</h2>

In [ ]:
product name
<span data-at="product_name" class="css-1pgnl76 eanm77i0" data-comp="StyledComponent BaseComponent ">Lotus Anti-Aging Night Moisturizer </span>



In [ ]:
price, multiple sizes

<div class="css-8llsux eanm77i0" data-comp="SwatchChiclet Chiclet StyledComponent BaseComponent "><span class="css-7b7t20">0.8 oz/ 25 mL</span></div>
<b class="css-0">$23.00</b>


<div class="css-8llsux eanm77i0" data-comp="SwatchChiclet Chiclet StyledComponent BaseComponent "><span class="css-7b7t20">1.7 oz/ 50 mL</span></div>
<b class="css-0">$45.00</b>

<div class="css-8llsux eanm77i0" data-comp="SwatchChiclet Chiclet StyledComponent BaseComponent "><span class="css-7b7t20">3.4 oz/ 100 mL<span class="css-1ibo83j eanm77i0" data-comp="Flag StyledComponent BaseComponent ">NEW</span></span></div>
<b class="css-0">$68.00</b>

# no. reviews
<span data-at="number_of_reviews" class="css-1coslxg">679</span>

# no. loves
<span class="css-jk94q9">58.5K</span>

# average rating
<span class="css-1ac1x0l eanm77i0" data-comp="StyledComponent BaseComponent ">4.4</span>

# % reccommended
<span class="css-1ac1x0l eanm77i0" data-comp="StyledComponent BaseComponent ">86%</span>

# similar products:
<img class="css-1rovmyu eanm77i0" src="https://www.sephora.com/productimages/sku/s1863604-main-zoom.jpg?imwidth=135&amp;pb=allure-best-2020&amp;imwidth=160" srcset="https://www.sephora.com/productimages/sku/s1863604-main-zoom.jpg?imwidth=135&amp;pb=allure-best-2020&amp;imwidth=160 1x, https://www.sephora.com/productimages/sku/s1863604-main-zoom.jpg?imwidth=135&amp;pb=allure-best-2020&amp;imwidth=320 2x" role="presentation" alt="" data-comp="Image StyledComponent BaseComponent ">

# ingredients:
<div class="css-1ue8dmw eanm77i0" data-comp="StyledComponent BaseComponent "><div>-Buckwheat Honey with Propolis and Royal Jelly Blend: Known for its soothing and humectant properties, this special honey blend provides skin-loving moisture and antioxidants.<br>-Fig Fruit Extract and Osmolytes: Bind and restore moisture, leaving skin looking plump and supple.<br>-Potent Ceramide Blend: Prevents moisture loss and replenishes the skin barrier.<br><br>Water/Aqua/Eau, Glycerin, Butyrospermum Parkii (Shea) Butter, Vegetable Oil/Olus Oil, C13-15 Alkane, Cetearyl Alcohol, Pentaerythrityl Tetraisostearate, 1,2-Hexanediol, Bis-Diglyceryl Polyacyladipate-1, Diisostearyl Malate, Xylitylglucoside, Honey Extract, Betaine, Panthenol, Phytosterols, Ceramide Np, Propolis Extract, Royal Jelly Extract, Tocopherol, Ficus Carica (Fig) Fruit Extract, Hippophae Rhamnoides Oil, Glucose, Bisabolol, Xylitol, Anhydroxylitol, Cetearyl Glucoside, Triolein, Acrylates/C10-30 Alkyl Acrylate Crosspolymer, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Arginine, Flavor (Aroma)*, Hydroxyacetophenone, Xanthan Gum, Glyceryl Dioleate, Sodium Dilauramidoglutamide Lysine, Sodium Phytate, Sorbitan Isostearate, Citric Acid, Potassium Sorbate, Sodium Benzoate.<br><br>*Natural Flavor/Arome Naturel.
<br><br><b>Clean at Sephora products are formulated without the following banned or restricted ingredients (please see Clean at Sephora landing page for full list of specific restrictions and allowances by category – this is not an exhaustive list): </b><br>
Acrylates, Aluminum Salts, Animal Musks/Fats/Oils, Benzophenones and related compounds, Butoxyethanol, Butylated hydroxyanisole (BHA), Butylated hydroxytulolene (BHT) under 0.1% , Carbon Black or Black 2, Coal Tar, Ethanolamines, Formaldehyde and Formaldehyde Releasing Agents, Hydroquinone, Lead and Lead Acetate, Mercury + Mercury Compounds (Thimerisol), Methoxyethanol, Methyl Cellosolve, Methylchloroisothiazolinone and Methylisothiazolinone, Mineral Oil, Nanomaterials (restrictions for specific nanomaterials only), Oxybenzone, Parabens, Petrolatum and Parrafin (USP grade only), Phenoxyethanol under 1%, Phthalates, Plastic Microbeads, Polyacrylamide &amp; Acrylamide, Resorcinol, Retinyl Palmitate, Styrene, Sulfates, Talc (no detection of asbestos) , Toluene, Triclosan and Triclocarban, Undisclosed Synthetic Fragrance (under 1% and formulated without Acetaldehyde, Acetone, Acetonitrile, Benzalkonium chloride, BPA, EDTA, Methylene chloride, PTFE, PFOA), 1, 4, Dioxane (ingredients must comply with impurity thresholds), Octinoxate, PFAS compounds, Nitromusks and Polycyclic Musks, Ethoxylated Ingredients (including PEGS or polyethylene glycol must meet contamination limits for 1’4 Dioxane and ethylene oxide), Cyclic Silicones, EDTA and derivatives (allowed if no technical substitute under 0.2%)</div></div>

